In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
import random
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings("ignore")

# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# algoritmos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# algoritmos de regresión
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# métricas
import sklearn.metrics as metrics

# hiperparametrización
from sklearn.model_selection import GridSearchCV

# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

import xgboost as xgb

from numpy import random
random.seed(42)

from sklearn.model_selection import train_test_split

In [3]:
file_path = "C:/Users/Herre/Desktop/Entrega_TFM/accounts_resultado_predicciones.csv"
file_path_2 = "C:/Users/Herre/Desktop/Entrega_TFM/investment_resultado_predicciones.csv"
file_path_3 = "C:/Users/Herre/Desktop/Entrega_TFM/financing_resultado_predicciones.csv"

In [5]:
df_accounts = pd.read_csv(file_path)
df_investments = pd.read_csv(file_path_2)
df_financing = pd.read_csv(file_path_3)

In [6]:
df_accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4076 entries, 0 to 4075
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_cid               4076 non-null   int64  
 1   prediccion           4076 non-null   int64  
 2   probabilidad_compra  4076 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 95.7 KB


In [7]:
df_investments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5015 entries, 0 to 5014
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_cid               5015 non-null   int64  
 1   prediccion           5015 non-null   int64  
 2   probabilidad_compra  5015 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 117.7 KB


In [8]:
df_financing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_cid               933 non-null    int64  
 1   prediccion           933 non-null    int64  
 2   probabilidad_compra  933 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 22.0 KB


In [9]:
# Seleccionamos solo las columnas relevantes: pk_cid y probabilidad_compra
df_accounts = df_accounts[['pk_cid', "probabilidad_compra"]]
df_investments = df_investments[['pk_cid', "probabilidad_compra"]]
df_financing = df_financing[['pk_cid', "probabilidad_compra"]]

# Renombramos las columnas de probabilidad de compra para que no haya conflictos al unir
df_accounts = df_accounts.rename(columns={"probabilidad_compra": 'probabilidad_accounts'})
df_investments = df_investments.rename(columns={"probabilidad_compra": 'probabilidad_investments'})
df_financing = df_financing.rename(columns={"probabilidad_compra": 'probabilidad_financing'})

# Unimos los dataframes por la columna pk_cid
df_combined = df_accounts.merge(df_investments, on='pk_cid', how='outer')
df_prediciones = df_combined.merge(df_financing, on='pk_cid', how='outer')

# Mostrar el dataframe combinado
df_prediciones.head()

,pk_cid,probabilidad_accounts,probabilidad_investments,probabilidad_financing
0,20243,NaN,0.980956,NaN
1,26018,NaN,0.991416,NaN
2,31624,NaN,0.980759,NaN
3,32263,NaN,0.946660,NaN
4,33046,NaN,0.948616,NaN


In [10]:
df_prediciones.head(20)

,pk_cid,probabilidad_accounts,probabilidad_investments,probabilidad_financing
0,20243,NaN,0.980956,NaN
1,26018,NaN,0.991416,NaN
2,31624,NaN,0.980759,NaN
3,32263,NaN,0.946660,NaN
4,33046,NaN,0.948616,NaN
5,46327,NaN,NaN,0.999841
6,47145,NaN,0.971060,NaN
7,51819,NaN,0.993509,NaN
8,55000,NaN,0.815528,NaN
9,56347,NaN,0.972121,NaN


In [11]:
#Imputaremos los nulos con la palabra "Unknown"
df_prediciones['probabilidad_accounts'].fillna(0, inplace=True)

In [12]:
df_prediciones['probabilidad_investments'].fillna(0, inplace=True)

In [13]:
df_prediciones['probabilidad_financing'].fillna(0, inplace=True)

In [14]:
# Redondear las columnas de probabilidad a 4 decimales
df_prediciones['probabilidad_accounts'] = df_prediciones['probabilidad_accounts'].round(4)
df_prediciones['probabilidad_investments'] = df_prediciones['probabilidad_investments'].round(4)
df_prediciones['probabilidad_financing'] = df_prediciones['probabilidad_financing'].round(4)

In [15]:
df_prediciones.head(20)

,pk_cid,probabilidad_accounts,probabilidad_investments,probabilidad_financing
0,20243,0.0,0.9810,0.0000
1,26018,0.0,0.9914,0.0000
2,31624,0.0,0.9808,0.0000
3,32263,0.0,0.9467,0.0000
4,33046,0.0,0.9486,0.0000
5,46327,0.0,0.0000,0.9998
6,47145,0.0,0.9711,0.0000
7,51819,0.0,0.9935,0.0000
8,55000,0.0,0.8155,0.0000
9,56347,0.0,0.9721,0.0000


In [16]:
# Guardar el DataFrame en un archivo CSV para revisarlo luego
df_prediciones.to_csv('Predicciones.csv', index=False)